# 네이버 영화평점

## 개요
- https://movie.naver.com/ 로 이동
- 평점/리뷰 메뉴 클릭

## 요청방식
- url
    - https://movie.naver.com/movie/point/af/list.nhn?&page=2
- 요청방식
    - get
- 요청파라미터
    - page: 페이지 번호
    
## 항목
- base : table.list_netizen > tbody > tr > td.title  : 이 td들 안에 다 있다.
- 영화제목 : > `a.movie : text`
    - href속성은 그 영화의 댓글만 모아놓은 url( 상대경로로 href="?st=mcode&sword=185917&target=after" 형식으로 쿼리 스트링만 있다.)
- 평점
    - > `div.list_netizen_score > em` 의 text
- 댓글
    - br의 next_sibling
    - text조회하면 안의 하위 태그들의 text도 조회된다.
        - **br을 찾은 뒤 그 다음 노드로 찾는다.**
            - tag.next_sibling : 텍스트 노드까지 포함 해서 다음 노드를 반환한다.
                - br다음의 text node를 조회할 것이므로 next_sibling 사용
            - tag.next_element : 다음 element node 반환
            
```html
<td class="title">
			
			
				<a href="?st=mcode&amp;sword=189633&amp;target=after" class="movie color_b">야구소녀</a>
			
			
			
			<div class="list_netizen_score">
				<span class="st_off"><span class="st_on" style="width:100%">별점 - 총 10점 중</span></span><em>10</em>
			</div>
			<br>기대없이 시청하는데 나도모르게 눈물을 흘렸네요 
			
			
			
				
				
				
				<a href="javascript:report('mkbk****', '3wfKF9kn6UVmVCmf5aXfMSpd8o/GJ1e3/VWl/7hODlk=', '기대없이 시청하는데 나도모르게 눈물을 흘렸네요', '17024530', 'point_after');" class="report" style="color:#8F8F8F" title="새 창">신고</a>
			
			
			</td>
```



### 1페이지 조회

In [3]:
import requests
from urllib import parse
from bs4 import BeautifulSoup
base_url = 'https://movie.naver.com/movie/point/af/list.nhn?&page={}'
url = base_url.format(1)
res=requests.get(url)

if res.status_code == 200:
    soup = BeautifulSoup(res.text)
    tds = soup.select('table.list_netizen > tbody > tr > td.title')
    print(len(tds))
    for td in tds:
        movie_title = td.select_one('a.movie').text.strip()
        link = td.select_one('a.movie').get('href')
        link = parse.urljoin(base_url, link)
        score = td.select_one('div.list_netizen_score > em').text.strip()
        comment = td.select_one('br').next_sibling.strip() #textnode 인 경우 text 로 조회할 필요 없다.
        print(movie_title, link, score, comment, sep=' :: ')
        print('-------------------------------------------------')

<Response [200]>
10
로드 무비 :: https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=34447&target=after :: 9 :: 
-------------------------------------------------
반도 :: https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=185917&target=after :: 1 :: 영화보다가 중간에 나오고싶었던적 처음입니다^^ 진짜 내인생 최악 억지감성과 초등학생이 쓴 시나리오같음 ㅋ
-------------------------------------------------
반도 :: https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=185917&target=after :: 7 :: 매드맥스랑 좀비영화를 섞은느낌? 재미있어요!
-------------------------------------------------
나를 차버린 스파이 :: https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=167602&target=after :: 9 :: 
-------------------------------------------------
강철비2: 정상회담 :: https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=188909&target=after :: 2 :: 배역선정도 너무 않좋고 스토리와 설정 진짜 최악이었습니다 정치같은건 모르겠고 졸다 나왔습니다 다큐도 아니고 진짜 재미없었어요 마지막 잠수함 전만 조금 재미있었네요
-------------------------------------------------
강철비2: 정상회담 :: https://movie.naver.com/movie/point/

In [7]:
# Guide 
# query string은 제거하고 붙인다.
url = 'http://www.naver.com/path/test?a=b&c=d'
print(parse.urljoin(url, '?kkk=10'))
print(parse.urljoin(url, '/abcdef'))
print(parse.urljoin(url,'my_path/test.html'))

http://www.naver.com/path/test?kkk=10
http://www.naver.com/abcdef
http://www.naver.com/path/my_path/test.html


## 전체 페이지 조회
- 끝페이지 조회하다 tds의 length가 0이면 조회한 것이 없는 것이다. 그렇게 체크하면된다. 그런데 너무 많으므로 100페이지정도만 조회한다.

In [23]:
import random
random.uniform(0.2, 1.2) #이 사이의 실수를 동일한 확률로 나오게 한다.

0.2934325558951632

In [ ]:
import requests
import time
import random
from bs4 import BeautifulSoup


base_url = 'https://movie.naver.com/movie/point/af/list.nhn?&page={}'
#결과 저장할 리스트
comment_list = []
for page in range(1, 101):
    url = base_url.format(page)
    res=requests.get(url)
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, 'lxml')
        tds = soup.select('table.list_netizen > tbody > tr > td.title')
#         if len(td): #마지막 페이지 까지 간 경우 조회결과가 없어 0이 된다. (반복문은 while True: 로 변환)
        for td in tds:
            movie_title = td.select_one('a.movie').text.strip()
            score = td.select_one('div.list_netizen_score > em').text.strip()
            comment = td.select_one('br').next_sibling.strip()
            # 리스트에 저장
            comment_list.append((movie_title, score, comment))
        interval = round(random.uniform(0.2, 1.2), 2)
        time.sleep(interval)
#         else:
#             break
print('종료')        

In [25]:
len(comment_list)

1000

In [35]:
import pandas as pd
df = pd.DataFrame(comment_list, 
                  columns=['영화제목','평점','댓글'])
df.shape

(1000, 3)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   영화제목    1000 non-null   object
 1   평점      1000 non-null   object
 2   댓글      1000 non-null   object
dtypes: object(3)
memory usage: 23.6+ KB


In [37]:
df.head()

,영화제목,평점,댓글
0,반도,8,넘넘넘 재밌어요~~!!!!
1,반도,1,48년을 살아오면서 지구력 인내력에는 자신 있었는데 반도는 나의 한계를 다시금 깨닫...
2,라이언 일병 구하기,10,
3,키싱 부스 2,10,재밋다 ㅠ 리 너무 존잘이야
4,반도,6,그냥 킬링타임용이네요.뭐 대단한거 없구요..
